# How to model a pool table particle system with Python and Pygame

This tutorial is made based on Peter Collingridge's tutorial: http://www.petercollingridge.co.uk/pygame-physics-simulation

I felt like his tutorial wasn't realistic enough, so I decided to slightly improve it. For collision I used the formulas found here: http://en.wikipedia.org/wiki/Elastic_collision#Two-_and_three-dimensional. I also added pool table-like pockets that you can use to "destroy" balls (i.e. remove them from the screen).

With that being said, let's look at how it works first: https://youtu.be/_-yY_xUYRio

And this is the code:

In [ ]:
import pygame
import random
import math

pygame.init()

BG_COLOR = (102, 140, 93)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
(SCREEN_WIDTH, SCREEN_HEIGHT) = (800, 600)
BALL_RADIUS = 20
POCKET_RADIUS = 23
DRAG = 0.999
ELASTICITY = 0.9

def collide(b1, b2):
    dx = b1.x - b2.x
    dy = b1.y - b2.y
        
    distance = math.hypot(dx, dy)
    if distance <= 2 * BALL_RADIUS:
        tangent = math.atan2(dy, dx) + math.pi / 2
        angle = tangent - math.pi / 2

        if b1.speed == 0:
            angle2 = math.atan2(math.sin(angle), 1 + math.cos(angle))
            angle1 = (math.pi - angle) / 2
            speed2 = (b2.speed * math.sqrt((1 + math.cos(angle)) / 2)) * ELASTICITY
            speed1 = (b2.speed * math.sin(angle / 2)) * ELASTICITY
        elif b2.speed == 0:
            angle1 = math.atan2(math.sin(angle), 1 + math.cos(angle))
            angle2 = (math.pi - angle) / 2
            speed1 = (b1.speed * math.sqrt((1 + math.cos(angle)) / 2)) * ELASTICITY
            speed2 = (b1.speed * math.sin(angle / 2)) * ELASTICITY
        else:
            vx1_after = b2.speed * math.cos(b2.angle - angle) * math.cos(angle) + b1.speed * math.sin(b1.angle - angle) * math.cos(angle + math.pi / 2)
            vy1_after = b2.speed * math.cos(b2.angle - angle) * math.sin(angle) + b1.speed * math.sin(b1.angle - angle) * math.sin(angle + math.pi / 2)
            vx2_after = b1.speed * math.cos(b1.angle - angle) * math.cos(angle) + b2.speed * math.sin(b2.angle - angle) * math.cos(angle + math.pi / 2)
            vy2_after = b1.speed * math.cos(b1.angle - angle) * math.sin(angle) + b2.speed * math.sin(b2.angle - angle) * math.sin(angle + math.pi / 2)
            angle1 = math.atan2(vy1_after, vx1_after)
            angle2 = math.atan2(vy2_after, vx2_after)
            speed1 = (vx1_after / math.cos(angle1)) * ELASTICITY
            speed2 = (vx2_after / math.cos(angle2)) * ELASTICITY
        
        (b1.angle, b1.speed) = (angle1, speed1)
        (b2.angle, b2.speed) = (angle2, speed2)

        overlap = 2 * BALL_RADIUS - distance + 1
        b1.x += math.cos(angle) * 0.5 * overlap
        b1.y += math.sin(angle) * 0.5 * overlap
        b2.x -= math.cos(angle) * 0.5 * overlap
        b2.y -= math.sin(angle) * 0.5 * overlap

def find_ball((mouse_x, mouse_y)):
    for ball in balls:
        if math.hypot(ball.x - mouse_x, ball.y - mouse_y) <= BALL_RADIUS:
            i = balls.index(ball)
            return balls[i]

class Ball():
    def __init__(self, (x, y), color):
        self.x = x
        self.y = y
        self.color = color
        self.speed = 0
        self.angle = 0

    def move(self):
        self.x += math.cos(self.angle) * self.speed
        self.y += math.sin(self.angle) * self.speed
        self.speed *= DRAG

    def bounce(self):
        if self.x > SCREEN_WIDTH - BALL_RADIUS:
            self.x = 2 * (SCREEN_WIDTH - BALL_RADIUS) - self.x
            self.angle = math.pi - self.angle
            self.speed *= ELASTICITY
        elif self.x < BALL_RADIUS:
            self.x = 2 * BALL_RADIUS - self.x
            self.angle = math.pi - self.angle
            self.speed *= ELASTICITY

        if self.y > SCREEN_HEIGHT - BALL_RADIUS:
            self.y = 2 * (SCREEN_HEIGHT - BALL_RADIUS) - self.y
            self.angle = -self.angle
            self.speed *= ELASTICITY
        elif self.y < BALL_RADIUS:
            self.y = 2 * BALL_RADIUS - self.y
            self.angle = -self.angle
            self.speed *= ELASTICITY

    def is_destroyed(self):
        for pocket in pockets:
            if self.x > pocket .x - BALL_RADIUS / 2 and self.x < pocket .x + BALL_RADIUS / 2 and self.y > pocket .y - BALL_RADIUS / 2 and self.y < pocket .y + BALL_RADIUS / 2:
                i = balls.index(self)
                del balls[i]

    def display(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), BALL_RADIUS)

class Pocket():
    def __init__(self, (x, y)):
        self.x = x
        self.y = y
        self.color = BLACK

    def display(self):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), POCKET_RADIUS)

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Pool table particle system')

balls = []
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), WHITE))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), BLACK))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (227, 205, 170)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (170, 192, 227)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (170, 227, 201)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (227, 170, 196)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (143, 50, 92)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (106, 50, 143)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (59, 47, 194)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (182, 194, 47)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (194, 113, 47)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (47, 128, 194)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (56, 92, 102)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (102, 56, 56)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (66, 242, 17)))
x = random.randint(2 * POCKET_RADIUS + BALL_RADIUS, SCREEN_WIDTH - 2 * POCKET_RADIUS - BALL_RADIUS)
y = random.randint(2 * BALL_RADIUS + BALL_RADIUS, SCREEN_HEIGHT - 2 * POCKET_RADIUS - BALL_RADIUS)
balls.append(Ball((x, y), (193, 17, 242)))

pockets = []
pockets.append(Pocket((POCKET_RADIUS, POCKET_RADIUS)))
pockets.append(Pocket((SCREEN_WIDTH - POCKET_RADIUS, POCKET_RADIUS)))
pockets.append(Pocket((POCKET_RADIUS, SCREEN_HEIGHT - POCKET_RADIUS)))
pockets.append(Pocket((SCREEN_WIDTH - POCKET_RADIUS, SCREEN_HEIGHT - POCKET_RADIUS)))
pockets.append(Pocket((SCREEN_WIDTH / 2, POCKET_RADIUS)))
pockets.append(Pocket((SCREEN_WIDTH / 2, SCREEN_HEIGHT - POCKET_RADIUS)))

selected_ball = None
mouse_coords = (0, 0)
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            (mouse_x, mouse_y) = pygame.mouse.get_pos()
            selected_ball = find_ball((mouse_x, mouse_y))
        elif event.type == pygame.MOUSEBUTTONUP:
            selected_ball = None

    if selected_ball:
        (mouse_x, mouse_y) = pygame.mouse.get_pos()
        dx = mouse_x - selected_ball.x
        dy = mouse_y - selected_ball.y
        selected_ball.angle = math.atan2(dy, dx)
        selected_ball.speed = math.hypot(dx, dy) * 0.1

    screen.fill(BG_COLOR)

    for pocket in pockets:
        pocket.display()

    for i, ball1 in enumerate(balls):
        ball1.move()
        ball1.bounce()
        for ball2 in balls[i + 1:]:
            collide(ball1, ball2)
        ball1.is_destroyed()
        ball1.display()

    pygame.display.flip()

pygame.quit()

Let's now look at how this code works. There are two classes, "Ball" (for balls) and "Pocket" (for pockets), as well as the collision detection function which defines how two balls will move after they have collided. Also, there is a function that will detect which ball is selected. Constant variables are named in capital letters, and arrays hold class instances.

We first import Python modules and initialize Pygame engine:

In [ ]:
import pygame
import random
import math

pygame.init()

Now we define several constants that we'll use in our program:

In [ ]:
BG_COLOR = (102, 140, 93)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
(SCREEN_WIDTH, SCREEN_HEIGHT) = (800, 600)
BALL_RADIUS = 20
POCKET_RADIUS = 23
DRAG = 0.999
ELASTICITY = 0.9

"BG_COLOR" defines background color, "DRAG" and "ELASTICITY" are essentially air friction and elasticity which will be used to reduce the speed of a ball over time and upon collision. The rest of the constants are self-explanatory.

Let's scroll down to this piece of code:

In [ ]:
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Pool table particle system')

It says: create a screen, set its size to the "(SCREEN_WIDTH, SCREEN_HEIGHT)" tuple and its caption to "Pool table particle system".

The code that goes right after those two lines, all the way down to the "while" loop, simply creates balls and pockets as instances of two classes, "Ball" and "Pocket". We set the coordinates of the balls to be random, so that they would be randomply placed on the pool table upon creation. Every ball object is appended to the "balls" array.

The pockets are created by specifying their location and then drawing them on the screen at that location.

Now we can move on to the main "while" loop. We need something to make the program stop running whenever we choose to, so let's create a variable "running" and set it to "True" for now:

In [ ]:
running = True

In our loop we first handle the events. We need a way to close the program and handle mouse clicks:

In [ ]:
for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            (mouse_x, mouse_y) = pygame.mouse.get_pos()
            selected_ball = find_ball((mouse_x, mouse_y))
        elif event.type == pygame.MOUSEBUTTONUP:
            selected_ball = None

Once we choose to quit, the "running" variable is assigned the "False" value and the program will exit.

Notice how on "MOUSEBUTTONDOWN" event we record the mouse position and send it to the function "find_ball" to see if we clicked on a ball or not. If we did, then the following condition is true and the code under it will run:

In [ ]:
if selected_ball:
        (mouse_x, mouse_y) = pygame.mouse.get_pos()
        dx = mouse_x - selected_ball.x
        dy = mouse_y - selected_ball.y
        selected_ball.angle = math.atan2(dy, dx)
        selected_ball.speed = math.hypot(dx, dy) * 0.1

It just calculates how far the mouse cursor got from when we clicked on the ball to when we released it. Then it will assign appropriate velocity to the ball.

Next the screen is filled with the background color, and pockets are on top of it.

Now for the most interesting and complicated part of the loop:

In [ ]:
for i, ball1 in enumerate(balls):
        ball1.move()
        ball1.bounce()
        for ball2 in balls[i + 1:]:
            collide(ball1, ball2)
        ball1.is_destroyed()
        ball1.display()

We first call the "move" method inside the "Ball" class for every ball we go through in our "balls" array. The method gives the ball speed and angle (i.e. velocity), and most calculations in those functions are based on the unit circle trigonometry: x = cos(angle), y = sin(angle), then we multiply those values by length (speed) to get how far and in what direction the ball travels. "DRAG" is just air friction:

In [ ]:
def move(self):
        self.x += math.cos(self.angle) * self.speed
        self.y += math.sin(self.angle) * self.speed
        self.speed *= DRAG

After we move the ball, we want to check if it collided with a border:

In [ ]:
def bounce(self):
        if self.x > SCREEN_WIDTH - BALL_RADIUS:
            self.x = 2 * (SCREEN_WIDTH - BALL_RADIUS) - self.x
            self.angle = math.pi - self.angle
            self.speed *= ELASTICITY
        elif self.x < BALL_RADIUS:
            self.x = 2 * BALL_RADIUS - self.x
            self.angle = math.pi - self.angle
            self.speed *= ELASTICITY

        if self.y > SCREEN_HEIGHT - BALL_RADIUS:
            self.y = 2 * (SCREEN_HEIGHT - BALL_RADIUS) - self.y
            self.angle = -self.angle
            self.speed *= ELASTICITY
        elif self.y < BALL_RADIUS:
            self.y = 2 * BALL_RADIUS - self.y
            self.angle = -self.angle
            self.speed *= ELASTICITY

"ELASTICITY" reduces speed if the ball did in fact collide with a border. We also change the angle to "reflect" the ball's direction after collision.

Now let's check if the ball collided with another ball (either moving or stationary):

In [ ]:
def collide(b1, b2):
    dx = b1.x - b2.x
    dy = b1.y - b2.y
        
    distance = math.hypot(dx, dy)
    if distance <= 2 * BALL_RADIUS:
        tangent = math.atan2(dy, dx) + math.pi / 2
        angle = tangent - math.pi / 2

        if b1.speed == 0:
            angle2 = math.atan2(math.sin(angle), 1 + math.cos(angle))
            angle1 = (math.pi - angle) / 2
            speed2 = (b2.speed * math.sqrt((1 + math.cos(angle)) / 2)) * ELASTICITY
            speed1 = (b2.speed * math.sin(angle / 2)) * ELASTICITY
        elif b2.speed == 0:
            angle1 = math.atan2(math.sin(angle), 1 + math.cos(angle))
            angle2 = (math.pi - angle) / 2
            speed1 = (b1.speed * math.sqrt((1 + math.cos(angle)) / 2)) * ELASTICITY
            speed2 = (b1.speed * math.sin(angle / 2)) * ELASTICITY
        else:
            vx1_after = b2.speed * math.cos(b2.angle - angle) * math.cos(angle) + b1.speed * math.sin(b1.angle - angle) * math.cos(angle + math.pi / 2)
            vy1_after = b2.speed * math.cos(b2.angle - angle) * math.sin(angle) + b1.speed * math.sin(b1.angle - angle) * math.sin(angle + math.pi / 2)
            vx2_after = b1.speed * math.cos(b1.angle - angle) * math.cos(angle) + b2.speed * math.sin(b2.angle - angle) * math.cos(angle + math.pi / 2)
            vy2_after = b1.speed * math.cos(b1.angle - angle) * math.sin(angle) + b2.speed * math.sin(b2.angle - angle) * math.sin(angle + math.pi / 2)
            angle1 = math.atan2(vy1_after, vx1_after)
            angle2 = math.atan2(vy2_after, vx2_after)
            speed1 = (vx1_after / math.cos(angle1)) * ELASTICITY
            speed2 = (vx2_after / math.cos(angle2)) * ELASTICITY
        
        (b1.angle, b1.speed) = (angle1, speed1)
        (b2.angle, b2.speed) = (angle2, speed2)

        overlap = 2 * BALL_RADIUS - distance + 1
        b1.x += math.cos(angle) * 0.5 * overlap
        b1.y += math.sin(angle) * 0.5 * overlap
        b2.x -= math.cos(angle) * 0.5 * overlap
        b2.y -= math.sin(angle) * 0.5 * overlap

This is just the physics of elastic collision, so you might want to check out the Wikipedia link I gave in the beginning.

What's "overlap" doing here? Well, the reason is that the speed with which the ball moves can actually exceed the speed of the screen update, so when the screen updates, two balls might have already "overlapped". The result of this would be balls "sticking together". So we need to get them "back" where they were before overlapping.

We're almost there, but we also need to check when the ball hits a pocket and is being "destroyed" (removed) as a consequence. This is pretty simple, just set some collision rules:

In [ ]:
def is_destroyed(self):
        for pocket in pockets:
            if self.x > pocket .x - BALL_RADIUS / 2 and self.x < pocket .x + BALL_RADIUS / 2 and self.y > pocket .y - BALL_RADIUS / 2 and self.y < pocket .y + BALL_RADIUS / 2:
                i = balls.index(self)
                del balls[i]

The last thing we need to do is display the ball and update the screen.

Outside the loop is the line required for Python IDE to "cleanup" after the program closes:

In [ ]:
pygame.quit()

If you don't add this line, Python shell will freeze.